# Kristins Alex 

## Imports

In [1]:
#pip install resnet

In [2]:
#pip install -U git+https://github.com/keras-team/keras git+https://github.com/keras-team/keras-applications

In [3]:
import os
from datetime import datetime
from dataclasses import dataclass
import warnings

import talos as ta
from talos.model import lr_normalizer
import numpy as np
from numpy.random import seed
import pandas as pd
import tensorflow as tf
import random

from tensorflow.compat.v1.keras import callbacks, backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import Model

available_gpus = tf.config.experimental.list_physical_devices('GPU')
built_with_cuda = tf.test.is_built_with_cuda()

if not (not available_gpus) & built_with_cuda:
    print("The installed version of TensorFlow {} includes GPU support.\n".format(tf.__version__))
    print("Num GPUs Available: ", len(available_gpus), "\n")
else:
    print("The installed version of TensorFlow {} does not include GPU support.\n".format(tf.__version__))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from numpy.random import seed
seed(1)
tf.random.set_seed(1)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.compat.v1.Session(config = config)
K.set_session(sess)

#gpu_options = tf.GPUOptions(allow_growth=True)
#session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Using TensorFlow backend.


The installed version of TensorFlow 2.1.0 includes GPU support.

Num GPUs Available:  1 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4134175289849093038
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 20264236482
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6192891619864284921
physical_device_desc: "device: 0, name: GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6"
]
1 Physical GPUs, 1 Logical GPUs


In [4]:
os.environ['PYTHONHASHSEED']='0'

s = 23
seed(s)
tf.random.set_seed(s)
random.seed(s)
np.random.seed(s)

## AlexNet

In [5]:
def alexnet(activation, leaky_alpha, dropout):
    
    K.clear_session()
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential([
        Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation=activation_layer, input_shape=(224,224,Global.num_image_channels)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Flatten(),
        Dense(4096, activation=activation_layer),
        Dropout(dropout),
        Dense(4096, activation=activation_layer),
        Dropout(dropout),
        Dense(units = 2, activation=activation_layer)
        #Dense(10, activation='softmax')
    ])
    return model

## VGG16

In [6]:
def vgg16_old(activation, leaky_alpha):
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    
    model.add(Conv2D(input_shape=(224,224,Global.num_image_channels), filters = 64, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Flatten())
    model.add(Dense(units = 4096, activation = activation_layer))
    model.add(Dense(units = 4096, activation = activation_layer))
    #model.add(Dense(units = 2, activation = "softmax"))
    model.add(Dense(units = 2, activation=activation_layer))

    #opt = Adam(lr = 0.001)
    #model.compile(optimizer = opt, loss= keras.losses.categorical_crossentropy, metrics = ['accuracy'])
    #model.summary()
    return model

In [7]:
def vgg16(activation, leaky_alpha):
    
    K.clear_session()
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    cnn = VGG16(include_top=False, weights=None, input_tensor=None, input_shape=(224, 224, Global.num_image_channels))
    for layer in cnn.layers:
        layer.trainable = True
    x = cnn.output

    x =  AveragePooling2D((7, 7), padding='same')(x)
    x = Flatten()(x)

    x = Dense(units = 512, activation = activation_layer)(x)
    x = Dense(units = 256, activation = activation_layer)(x)
    x = Dense(units = 64, activation = activation_layer)(x)
    x = Dense(units = 2, activation = activation_layer)(x)
    model = Model(cnn.input, x)

    return model

## ResNet

In [8]:
def resnet(activation, leaky_alpha):
    
    K.clear_session()
         
    activation_layer = ReLU()
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
        
    model = Sequential()
    cnn = ResNet50(include_top=False, weights=None, input_tensor=None, input_shape=(224, 224, Global.num_image_channels))
    for layer in cnn.layers:
        layer.trainable = True
    x = cnn.output

    x =  AveragePooling2D((7, 7), padding='same')(x)
    x = Flatten()(x)

    x = Dense(units = 512, activation = activation_layer)(x)
    x = Dense(units = 256, activation = activation_layer)(x)
    x = Dense(units = 64, activation = activation_layer)(x)
    x = Dense(units = 2, activation = activation_layer)(x)
    model = Model(cnn.input, x)

    return model

## Generation of the DNN

In [9]:
def gen_net(x_train, y_train, x_val, y_val, params):
    
    K.clear_session()
    
    train_generator, valid_generator = create_data_pipline(params['batch_size'], params['samples'])
    tg_steps_per_epoch = train_generator.n // train_generator.batch_size
    vg_validation_steps = valid_generator.n // valid_generator.batch_size
    print('Steps per Epoch: {}, Validation Steps: {}'.format(tg_steps_per_epoch, vg_validation_steps))
    
 
    if(Global.net_architecture == 'ALEX'):
        model = alexnet(params['activation'], params['leaky_alpha'], params['dropout'])
    elif(Global.net_architecture =='VGG16'):
        model = vgg16(params['activation'], params['leaky_alpha'])
    elif(Global.net_architecture == 'RESNET'):
        model = resnet(params['leaky_alpha'], params['leaky_alpha'])
    else:
        print('Wrong net architecture!')
            
    model.compile(
        optimizer = params['optimizer'](lr = lr_normalizer(params['lr'], params['optimizer'])), 
        loss = Global.loss_function, 
        metrics = get_reduction_metric(Global.reduction_metric)
    )
    #print(model.summary())
    print('_________________________________________________________________')
    
    startTime = datetime.now()
    
    out = model.fit(
        x = train_generator,
        epochs = params['epochs'],
        validation_data = valid_generator,
        steps_per_epoch = tg_steps_per_epoch,
        validation_steps = vg_validation_steps,
        workers = 8
    )
    print("Time taken:", datetime.now() - startTime)

    return out, model

### Benutzerdefinierte Kostenfunktion & Metrik

In [10]:
def mean_absolut_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis = -1)

### Hilfsfunktion

In [11]:
def get_reduction_metric(metric):
    
    if metric == 'mean_absolut_error':
        return [mean_absolut_error]
    else:
        assert(False, 'Metric yet unknown - Please modify get_Reduction_Metric to meet your requirements')
        return None

### Struct for global parameter

In [12]:
@dataclass
class global_parameter:
    loss_function: str = 'mean_squared_error'
    reduction_metric: str = 'mean_absolut_error'
    monitor_value: str = 'val_mean_absolut_error'
        
    net_architecture = 'VGG16' # 'ALEX' vs 'VGG16' vs 'RESNET'
    
    dataset: str = '201129_2031'
    device: str = 'RTX_2080_Ti'
    data_augmentation: bool = True
    image_channels: str = 'rgb' # just change this, everything else will automaticlly adjusted
    num_image_channels: int = 3
    image_dir: str = '..\\..\\data_generation\\dataset\\{}\\'.format(dataset)
    
    csv_file_name: str = 'labels_ks_RGB.csv'
    csv_file: str = image_dir + csv_file_name
    target_dir: str = '..\\output\\{}_{}_{}\\'.format(net_architecture, dataset, image_channels)
    results: str = '\\..\\{}_{}_Results.csv'.format(net_architecture, dataset)

        
Global = global_parameter

if(Global.image_channels == 'rgba'):
    Global.num_image_channels = 4
    csv_file_name: str = 'lables_ks_RGBD.csv'
    csv_file: str = image_dir + csv_file_name
    target_dir: str = '..\\output\\{}_{}_{}\\'.format(net_architecture, dataset, image_channels)
    results: str = '\\..\\{}_{}_Results.csv'.format(net_architecture, dataset)

### Generierung Datenpipeline (Angepasst für Talos)

In [13]:
def create_data_pipline(batch_size, num_samples):
    
    
    df = pd.read_csv(Global.csv_file)
    df_shuffled = df.sample(frac = 1, random_state = 1)
    df_train = df_shuffled[0 : int(num_samples * 0.8 // batch_size * batch_size)]
    df_valid = df_shuffled.drop(df_shuffled.index[0 : df_train.shape[0]])[0 : int(num_samples * 0.2 // batch_size * batch_size)]
    
    if Global.data_augmentation:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.1,
            brightness_range = (0.5, 1.0), 
            fill_mode = 'nearest'
        )
    else:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255
        )
        
    train_generator = train_data_generator.flow_from_dataframe(
        dataframe = df_train,
        directory = Global.image_dir,
        x_col = 'Filename',
        y_col = ['Elevation', 'Azimuth'],
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = Global.image_channels,
        shuffle = True,
        seed = 77,
        batch_size = batch_size
    )
        
    valid_data_generator = ImageDataGenerator(
        rescale = 1./255
    )
    
    valid_generator = valid_data_generator.flow_from_dataframe(
        dataframe = df_valid,
        directory = Global.image_dir,
        x_col = 'Filename',
        y_col = ['Elevation', 'Azimuth'],
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = Global.image_channels,
        shuffle = False,
        seed = 77,
        batch_size = batch_size
    )
    
    return train_generator, valid_generator

In [14]:

if(not os.path.exists(Global.target_dir)):
    os.makedirs(Global.target_dir)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(Global.target_dir))

device_file = open(Global.target_dir + '{}.txt'.format(Global.device), "a+")
device_file.close()

### GridSerach Parameter

In [15]:
#     Adam = RMSprop + Momentum (lr=0.001)
#     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
#     RMSprop = (lr=0.001)
#     SGD = (lr=0.01)
#     Adagrad
#
#hyper_parameter = {
#    'samples': [20000],
#    'epochs': [1],
#    'batch_size': [32, 64],
#    'optimizer': [Adam],
#    'lr': [1, 2, 5],
#    'first_neuron': [1024, 2048, 4096],
#    'dropout': [0.25, 0.50],
#    'activation': ['leakyrelu', 'relu'],
#    'hidden_layers': [0, 1, 2, 3, 4],
#    'leaky_alpha': [0.1] #Default bei LeakyReLU, sonst PReLU
#}

hyper_parameter = {
    'samples': [20000],
    'epochs': [1],
    'batch_size': [64], #32],# 64],
    'optimizer': [Adam],
    'lr': [1], #1, 2, 5],
    'first_neuron':  [1024, 2048, 4096],
    'dropout': [0.25, 0.50],
    'activation': ['relu', 'leakyrelu'],
    'hidden_layers': [0, 1, 2, 3, 4],
    'leaky_alpha': [0.1] #Default bei LeakyReLU, sonst PReLU
}

### Start Talos Search

In [16]:
dummy_x = np.empty((1, 2, 3, 224, 224))
dummy_y = np.empty((1, 2))

with tf.device('/device:GPU:0'):
    with warnings.catch_warnings():
            warnings.simplefilter("ignore")
    
            t = ta.Scan(
                x = dummy_x,
                y = dummy_y,
                model = gen_net,
                params = hyper_parameter,
                experiment_name = '{}'.format(Global.dataset),
                #shuffle=False,
                reduction_metric = Global.reduction_metric,
                disable_progress_bar = False,
                print_params = True,
                clear_session = True,
                save_weights = False
            )
        

#t.data.to_csv(Global.target_dir + Global.results, index = True)
t.data.to_csv(Global.target_dir + Global.results, index = True, mode='a', header=False)

  0%|                                                                                           | 0/60 [00:00<?, ?it/s]

{'activation': 'relu', 'batch_size': 64, 'dropout': 0.25, 'epochs': 1, 'first_neuron': 1024, 'hidden_layers': 0, 'leaky_alpha': 0.1, 'lr': 1, 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'samples': 20000}
Found 16000 validated image filenames.
Found 3968 validated image filenames.
Steps per Epoch: 250, Validation Steps: 62
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 250 steps, validate for 62 steps
250/250 [==============================] - 1263s 5s/step - loss: 9720.2834 - mean_absolut_error: 69.7432 - val_loss: 7989.6884 - val_mean_absolut_error: 63.9244


  2%|█▎                                                                            | 1/60 [21:06<20:45:18, 1266.41s/it]

Time taken: 0:21:04.663854
{'activation': 'relu', 'batch_size': 64, 'dropout': 0.25, 'epochs': 1, 'first_neuron': 1024, 'hidden_layers': 1, 'leaky_alpha': 0.1, 'lr': 1, 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'samples': 20000}
Found 16000 validated image filenames.
Found 3968 validated image filenames.
Steps per Epoch: 250, Validation Steps: 62
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 250 steps, validate for 62 steps
250/250 [==============================] - 172s 687ms/step - loss: 10661.7422 - mean_absolut_error: 71.9648 - val_loss: 6828.0659 - val_mean_absolut_error: 59.9875
Time taken: 0:02:53.501997


  3%|██▋                                                                            | 2/60 [24:01<15:07:38, 938.95s/it]

...

 82%|████████████████████████████████████████████████████████████████▌              | 49/60 [2:41:47<32:17, 176.09s/it]

{'activation': 'leakyrelu', 'batch_size': 64, 'dropout': 0.5, 'epochs': 1, 'first_neuron': 1024, 'hidden_layers': 4, 'leaky_alpha': 0.1, 'lr': 1, 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'samples': 20000}
Found 16000 validated image filenames.
Found 3968 validated image filenames.
Steps per Epoch: 250, Validation Steps: 62
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 250 steps, validate for 62 steps
250/250 [==============================] - 176s 705ms/step - loss: 17031.8169 - mean_absolut_error: 93.8877 - val_loss: 20033.6817 - val_mean_absolut_error: 104.3911
Time taken: 0:02:57.794551


 83%|█████████████████████████████████████████████████████████████████▊             | 50/60 [2:44:46<29:31, 177.11s/it]

{'activation': 'leakyrelu', 'batch_size': 64, 'dropout': 0.5, 'epochs': 1, 'first_neuron': 2048, 'hidden_layers': 0, 'leaky_alpha': 0.1, 'lr': 1, 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'samples': 20000}
Found 16000 validated image filenames.
Found 3968 validated image filenames.
Steps per Epoch: 250, Validation Steps: 62
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 250 steps, validate for 62 steps
250/250 [==============================] - 173s 694ms/step - loss: 16474.4141 - mean_absolut_error: 91.6518


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  MemoryError: Unable to allocate 36.8 MiB for an array with shape (64, 224, 224, 3) and data type float32
Traceback (most recent call last):

  File "D:\anaconda\envs\tf_ks\lib\site-packages\tensorflow_core\python\ops\script_ops.py", line 236, in __call__
    ret = func(*args)

  File "D:\anaconda\envs\tf_ks\lib\site-packages\tensorflow_core\python\data\ops\dataset_ops.py", line 789, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "D:\anaconda\envs\tf_ks\lib\site-packages\tensorflow_core\python\keras\engine\data_adapter.py", line 975, in generator_fn
    yield x[i]

  File "D:\anaconda\envs\tf_ks\lib\site-packages\keras_preprocessing\image\iterator.py", line 65, in __getitem__
    return self._get_batches_of_transformed_samples(index_array)

  File "D:\anaconda\envs\tf_ks\lib\site-packages\keras_preprocessing\image\iterator.py", line 222, in _get_batches_of_transformed_sam